In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import (t, f, norm)

In [10]:
data = pd.read_csv("../data/nhanes_2015_2016.csv", usecols = ["SMQ020", "RIAGENDR", "DMDCITZN"]).dropna(axis = 0)

In [13]:
data.isna().sum(axis = 0)

SMQ020      0
RIAGENDR    0
DMDCITZN    0
dtype: int64

In [14]:
# Population proportion.

# Null hypothesis: Population proportion of smokers in 0.4
# Alternative hypothesis: Population proportion of smokers is not 0.4

In [15]:
data.SMQ020.unique()

array([1, 2, 7, 9], dtype=int64)

In [16]:
# best estimate, sample proportion of smokers.

best_est = (data.SMQ020 == 1).mean()
best_est

0.40442971747471224

In [17]:
# estimated standard error

stderr_null = np.sqrt((0.4 * (1 - 0.4)) / data.shape[0])
stderr_null

0.006469590264171095

In [18]:
# test statistic

zstat = (best_est - 0.4) / data.shape[0]
zstat

7.72535311250822e-07

In [21]:
p = 2 * (1 - norm.cdf(zstat))
p

0.9999993836060024

In [23]:
# p value here is very larger compared to the significance level of 0.05
# So, we cannot reject the Null hypothesis,
# We have enough evidence to claim that the population proportion of smokers is indeed 0.40